In [ ]:
#pip'i yükselt ve ultralytics (YOLOv8)'i yükle
!pip install --upgrade pip
!pip install ultralytics

#İsteğe bağlı: GPU'yu kontrol et
import torch
print("CUDA kullanılabilir:", torch.cuda.is_available())
!nvidia-smi

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="1NeDcIkHadlAytbTer2B")
project = rf.workspace("bitirmeodevi-b1usn").project("bitime-projesi-j58xp")
version = project.version(3)
dataset = version.download("yolov8")

In [ ]:
from ultralytics import YOLO

#Yolları ayarla
dataset_path = '/content/bitime-projesi-3/data.yaml'

#Önceden eğitilmiş YOLOv8n (nano) modelini yükle
model = YOLO('yolov8n.pt')

#Aşırı öğrenmeye karşı önlemlerle birlikte eğit ve grafikler oluştur
model.train(
    data=dataset_path,             #dataset.yaml dosyasının yolu
    epochs=50,                     #50 epoch ile başla
    imgsz=640,                     #görsel boyutu
    batch=8,                       #GPU dostu batch boyutu
    device=0,                      #GPU T4
    name='yolov8-model',           #deney adı
    augment=True,                  #veri artırmayı etkinleştir
    patience=10,                   #iyileşme olmazsa erken durdurma
    pretrained=True,               #önceden eğitilmiş ağırlıkları ince ayarla
    plots=True                     #otomatik olarak grafik oluştur
)

In [ ]:
#En iyi eğitilmiş modeli yükle
model = YOLO('/content/runs/detect/yolov8-model2/weights/best.pt')

#Test görselleri klasöründe çıkarım (inference) çalıştır
results = model.val(data='/content/bitime-projesi-3/data.yaml', split='test')

In [ ]:
#Eğitilmiş YOLOv8 modelini yükle
model = YOLO('/content/runs/detect/yolov8-model2/weights/best.pt')

#Yeni video üzerinde çıkarım (inference) çalıştır
results = model.predict(
    source='/content/bitime-projesi-3/red-ball.mp4',
    save=True,        # açıklamalı videoyu kaydet
    conf=0.25,        # güven eşiği
    device=0          # GPU
)

print("Inference tamamlandı. Kaydedilen videoyu runs/detect/ klasöründe kontrol et")

In [ ]:
#YOLO modelini yükle
import cv2
import pandas as pd
from ultralytics import YOLO

#YOLO modelini yükle
model = YOLO("/content/best.pt")

#Girdi ve çıktı yolları
video_path = "/content/colored-ball.mp4"
output_video = "/content/colored-ball_annotated.mp4"
csv_path = "/content/ball_coordinates.csv"

#Videoyu aç
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#Video yazıcı
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

#CSV depolama
data = []
frame_idx = 0

#Videoyu işle
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    #Bu karede YOLO çıkarımı çalıştır
    results = model.predict(frame, verbose=False, conf=0.5)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            xyxy = box.xyxy[0].tolist()  # [x1, y1, x2, y2]
            conf = float(box.conf[0])
            x_center = (xyxy[0] + xyxy[2]) / 2
            y_center = (xyxy[1] + xyxy[3]) / 2

            #Normalize et
            x_norm = x_center / frame_width
            y_norm = y_center / frame_height
            y_norm_math = 1 - y_norm
            data.append([frame_idx, x_norm, y_norm, y_norm_math, conf])

            #Kutu çiz
            x1, y1, x2, y2 = map(int, xyxy)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"Top ({x_norm:.2f},{y_norm_math:.2f})"
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 158), 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()

#CSV'yi kaydet
df = pd.DataFrame(data, columns=["frame", "x_norm", "y_norm", "y_norm_flipped", "confidence"])
df.to_csv(csv_path, index=False)

print(f"Açıklamalı video kaydedildi: {output_video}")
print(f"Koordinat CSV dosyası kaydedildi: {csv_path}")


In [ ]:
#----------------------------
#1 Imports
#----------------------------
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#----------------------------
#2 CSV'yi yükle
#----------------------------
csv_path = "/content/ball_coordinates.csv"
df = pd.read_csv(csv_path)

# Tutarlılık için ters çevrilmiş Y koordinatını kullan
coords = df[['x_norm','y_norm_flipped']].values

#----------------------------
#3 Normalleştir
#----------------------------
scaler = MinMaxScaler()
coords_scaled = scaler.fit_transform(coords)

#----------------------------
#4 Dizileri hazırla
#----------------------------
SEQ_LENGTH = 10
PRED_AHEAD = 5

def create_sequences(data, seq_length=SEQ_LENGTH, pred_ahead=PRED_AHEAD):
    X, y = [], []
    for i in range(len(data) - seq_length - pred_ahead):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length+pred_ahead-1])
    return np.array(X), np.array(y)

X, y = create_sequences(coords_scaled)
print(f"X şekli: {X.shape}, y şekli: {y.shape}")

#----------------------------
#5 Eğitme/teste ayır
#----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False  # zamansal sıralamayı koru
)
print(f"Eğitim: {X_train.shape}, Test: {X_test.shape}")

#----------------------------
#6 LSTM modelini oluştur
#----------------------------
model = Sequential([
    LSTM(64, input_shape=(SEQ_LENGTH, 2), return_sequences=True),
    Dropout(0.3),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(2, activation='linear')
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

#----------------------------
#7 Erken durdurma
#----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

#----------------------------
#8 Eğit
#----------------------------
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=2
)

#----------------------------
#9 Modeli kaydet
#----------------------------
model.save("trained_lstm_ball.keras")
print("Model 'trained_lstm_ball.keras' olarak kaydedildi")

#----------------------------
#10 Test et / Test kümesinde değerlendir
#----------------------------
y_pred = model.predict(X_test)

# Orijinal koordinatlara dönüştür
y_test_orig = scaler.inverse_transform(y_test)
y_pred_orig = scaler.inverse_transform(y_pred)

#----------------------------
#11 X ve Y tahminlerini ayrı ayrı çiz
#----------------------------
plt.figure(figsize=(10,5))
plt.plot(y_test_orig[:,0], label="Gerçek X")
plt.plot(y_pred_orig[:,0], label="Tahmin X", linestyle='--')
plt.legend()
plt.title("X-koordinatı: Gerçek vs Tahmin (Test Kümesi)")
plt.xlabel("Örnek indeksi")
plt.ylabel("X (orijinal ölçek)")
plt.show()

plt.figure(figsize=(10,5))
plt.plot(y_test_orig[:,1], label="Gerçek Y")
plt.plot(y_pred_orig[:,1], label="Tahmin Y", linestyle='--')
plt.legend()
plt.title("Y-koordinatı: Gerçek vs Tahmin (Test Kümesi)")
plt.xlabel("Örnek indeksi")
plt.ylabel("Y (orijinal ölçek)")
plt.show()

#----------------------------
#12 2D Yörünge
#----------------------------
plt.figure(figsize=(6,6))
plt.plot(y_test_orig[:,0], y_test_orig[:,1], 'g', label="Gerçek yörünge")
plt.plot(y_pred_orig[:,0], y_pred_orig[:,1], 'r--', label="Tahmin yörünge")
plt.legend()
plt.title("Top Yörüngesi: Gerçek vs Tahmin (Test Kümesi)")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

#----------------------------
#13 Doğruluk / Hata Metrikleri
#----------------------------
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae_total = mean_absolute_error(y_test_orig, y_pred_orig)
mse_total = mean_squared_error(y_test_orig, y_pred_orig)

# Yaklaşık % doğruluk (normalleştirilmiş koordinatlar 0-1 aralığında)
accuracy_pct_total = (1 - mae_total / 1.0) * 100

# Eksen başına MAE ve % doğruluk
mae_x = mean_absolute_error(y_test_orig[:,0], y_pred_orig[:,0])
mae_y = mean_absolute_error(y_test_orig[:,1], y_pred_orig[:,1])
accuracy_x = (1 - mae_x) * 100
accuracy_y = (1 - mae_y) * 100

print(f"Toplam MAE: {mae_total:.4f}, Toplam MSE: {mse_total:.4f}, Genel Doğruluk: {accuracy_pct_total:.2f}%")
print(f"X-ekseni MAE: {mae_x:.4f}, Doğruluk: {accuracy_x:.2f}%")
print(f"Y-ekseni MAE: {mae_y:.4f}, Doğruluk: {accuracy_y:.2f}%")


In [ ]:

# ----------------------------
# Imports
# ----------------------------
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# ----------------------------
# Dosya Yolları ve Parametreler
# ----------------------------
video_path = "/content/test-ball.mp4"
yolo_weights = "/content/best.pt"
lstm_model_path = "/content/trained_lstm_ball.keras"
csv_path = "/content/ball_coordinates.csv"
annotated_video_path = "/content/video_detected_and_predicted.avi"

SEQ_LENGTH = 10
CONF_THRESHOLD = 0.5

# ----------------------------
# Modelleri Yükle
# ----------------------------
try:
    yolo_model = YOLO(yolo_weights)
    lstm_model = load_model(lstm_model_path)
except Exception as e:
    print(f"Modeller yüklenirken hata: {e}")
    exit()

# ----------------------------
# Video Aç ve Veri Yapılarını Hazırla
# ----------------------------
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Video açılamadı: {video_path}")
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

data = []            # CSV için veri
coords_list = []     # LSTM için normalleştirilmiş [x, 1-y] verileri
detection_map = {}   # PASS 2 için kutu ve etiket verisi
frame_idx = 0

# ----------------------------
# PASS 1: YOLO ile Top Tespiti ve Veri Toplama
# ----------------------------
print("1. AŞAMA: Top tespiti ve koordinat toplama...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_width = frame.shape[1]
    frame_height = frame.shape[0]
    results = yolo_model.predict(frame, verbose=False, conf=CONF_THRESHOLD)

    for result in results:
        boxes = result.boxes
        if len(boxes) > 0:
            box = boxes[0]
            xyxy = box.xyxy[0].tolist()
            conf = float(box.conf[0])
            x1, y1, x2, y2 = map(int, xyxy)

            # Normalleştirilmiş koordinatları hesapla
            x_center = (x1 + x2) / 2
            y_center = (y1 + y2) / 2
            x_norm = x_center / frame_width
            y_norm = y_center / frame_height
            y_norm_flipped = 1 - y_norm  # 0=alt, 1=üst (fiziksel koordinat sistemi)

            # LSTM ve CSV için veri kaydet
            coords_list.append([x_norm, y_norm_flipped])
            data.append([frame_idx, x_norm, y_norm, y_norm_flipped, conf])

            # PASS 2 için kutu verisini sakla
            detection_map[frame_idx] = {
                'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2,
                'x_norm': x_norm, 'y_norm': y_norm, 'conf': conf
            }
            break

    frame_idx += 1

cap.release()

# CSV kaydet
df = pd.DataFrame(data, columns=["frame", "x_norm", "y_norm", "y_norm_flipped", "confidence"])
df.to_csv(csv_path, index=False)
print(f"CSV kaydedildi: {csv_path} (Toplam tespit edilen frame: {len(data)})")

# ----------------------------
# LSTM ile Yörünge Tahmini
# ----------------------------
if len(coords_list) < SEQ_LENGTH:
    print(f"Yeterli veri yok ({len(coords_list)} nokta). Tahmin atlanıyor.")
    predicted_points = []
else:
    coords_scaled = np.array(coords_list)
    scaler = MinMaxScaler()
    coords_scaled = scaler.fit_transform(coords_scaled)

    predicted_points_scaled = []
    print("Gelecek yörünge tahmini hesaplanıyor...")

    for i in range(len(coords_scaled) - SEQ_LENGTH):
        seq = coords_scaled[i:i+SEQ_LENGTH].reshape(1, SEQ_LENGTH, 2)
        pred = lstm_model.predict(seq, verbose=0)
        predicted_points_scaled.append(pred[0])

    predicted_points_scaled = np.array(predicted_points_scaled)
    predicted_points = scaler.inverse_transform(predicted_points_scaled)
    print(f"Tahmin tamamlandı ({len(predicted_points)} adım tahmin edildi).")

# ----------------------------
# PASS 2: Video Üzerine Tespit ve Tahmin Çizimi
# ----------------------------
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(annotated_video_path, fourcc, fps, (frame_width, frame_height))
frame_idx = 0

print("2. AŞAMA: Video üzerine tespit ve tahmin çiziliyor...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_idx in detection_map:
        det = detection_map[frame_idx]
        x1, y1, x2, y2 = det['x1'], det['y1'], det['x2'], det['y2']
        x_norm, y_norm, conf = det['x_norm'], det['y_norm'], det['conf']

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"TESPİT EDİLDİ: Top ({x_norm:.2f},{y_norm:.2f}) G:{conf:.2f}"
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    prediction_index = frame_idx - SEQ_LENGTH
    if 0 <= prediction_index < len(predicted_points):
        predicted_x_norm = predicted_points[prediction_index][0]
        predicted_y_flipped = predicted_points[prediction_index][1]

        x_pred = int(predicted_x_norm * frame_width)
        y_pred = int((1 - predicted_y_flipped) * frame_height)

        cv2.circle(frame, (x_pred, y_pred), 8, (0, 165, 255), -1)
        if prediction_index > 0:
            prev_x = predicted_points[prediction_index - 1][0]
            direction = "LEFT" if predicted_x_norm < prev_x else "RIGHT"
        else:
            direction = "UNKNOWN"
        pred_label = f"TAHMİN: ({predicted_x_norm:.2f},{predicted_y_flipped:.2f} TERS)"
        cv2.putText(frame, pred_label, (x_pred + 10, y_pred - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 165, 255), 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print(f"Video üzerine tespit ve tahmin kaydedildi: {annotated_video_path}")

# ----------------------------
# Metrikler ve Grafikler
# ----------------------------
if len(predicted_points) > 0:
    y_test_orig = np.array(coords_list[SEQ_LENGTH:len(predicted_points)+SEQ_LENGTH])
    y_pred_orig = np.array(predicted_points)

    mse_total = mean_squared_error(y_test_orig, y_pred_orig)
    mae_total = mean_absolute_error(y_test_orig, y_pred_orig)
    accuracy_pct_total = (1 - mae_total / 1.0) * 100
    mae_x = mean_absolute_error(y_test_orig[:,0], y_pred_orig[:,0])
    mae_y = mean_absolute_error(y_test_orig[:,1], y_pred_orig[:,1])
    accuracy_x = (1 - mae_x) * 100
    accuracy_y = (1 - mae_y) * 100

    print("\nTahmin Metrikleri:")
    print(f"Toplam MAE: {mae_total:.4f}, Toplam MSE: {mse_total:.4f}, Genel Doğruluk: {accuracy_pct_total:.2f}%")
    print(f"X ekseni MAE: {mae_x:.4f}, Doğruluk: {accuracy_x:.2f}%")
    print(f"Y ekseni MAE: {mae_y:.4f}, Doğruluk: {accuracy_y:.2f}%")

    plt.figure(figsize=(10,6))
    plt.plot(y_test_orig[:,0], y_test_orig[:,1], label="Gerçek Yörünge", marker='o', markersize=3)
    plt.plot(y_pred_orig[:,0], y_pred_orig[:,1], label="Tahmin Edilen Yörünge", marker='x', markersize=3)
    plt.gca().invert_yaxis()
    plt.xlabel("Normalleştirilmiş X")
    plt.ylabel("Normalleştirilmiş Y (Ters)")
    plt.title("Top Yörüngesi: Gerçek vs Tahmin")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("/content/yol_karşılaştırma.png")
    plt.show()

    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(y_test_orig[:,0], label="Gerçek X", marker='o', markersize=3)
    plt.plot(y_pred_orig[:,0], label="Tahmin X", marker='x', markersize=3)
    plt.title("X Koordinatı: Gerçek vs Tahmin")
    plt.xlabel("Frame İndeksi")
    plt.ylabel("Normalleştirilmiş X")
    plt.legend()
    plt.grid(True)

    plt.subplot(1,2,2)
    plt.plot(y_test_orig[:,1], label="Gerçek Y", marker='o', markersize=3)
    plt.plot(y_pred_orig[:,1], label="Tahmin Y", marker='x', markersize=3)
    plt.title("Y Koordinatı: Gerçek vs Tahmin")
    plt.xlabel("Frame İndeksi")
    plt.ylabel("Normalleştirilmiş Y (Ters)")
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig("/content/xy_karşılaştırma.png")
    plt.show()
else:
    print("Değerlendirilecek tahmin bulunamadı.")
